<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>


# Deep Learning Basics with PyTorch

**Dr. Yves J. Hilpisch with GPT-5**


# Chapter 6 — Building Blocks of Neural Networks
Colab-ready notebook covering neurons, activations, manual forward passes, and XOR decision boundaries.

In [ ]:
# !pip -q install torch numpy matplotlib scikit-learn
import torch, numpy as np, matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8') # plotting
%config InlineBackend.figure_format = 'retina'


## Activation functions

In [ ]:
x = torch.linspace(-4, 4, steps = 9)
torch.sigmoid(x), torch.tanh(x), torch.relu(x)


In [ ]:
x = np.linspace(-6, 6, 400)
plt.figure(figsize = (5, 3)) # plotting
plt.plot(x, 1/(1+np.exp(-x)), label = 'sigmoid') # plotting
plt.plot(x, np.tanh(x), label = 'tanh') # plotting
plt.plot(x, np.maximum(0, x), label = 'ReLU') # plotting
plt.legend(frameon = False) # plotting
plt.tight_layout() # plotting
plt.show() # plotting


## Manual forward for a 2-layer MLP

In [ ]:
torch.manual_seed(0) # reproducibility
x = torch.randn(5, 2)
W1 = torch.randn(2, 4) # layer 1 weights # layer 1 weights  # layer 1 weights
b1 = torch.randn(4) # layer 1 bias # layer 1 bias  # layer 1 bias
W2 = torch.randn(4, 1) # layer 2 weights # layer 2 weights  # layer 2 weights
b2 = torch.randn(1) # layer 2 bias # layer 2 bias  # layer 2 bias
h = torch.relu(x @ W1 + b1) # hidden activations  # hidden activations
y = h @ W2 + b2 # targets/labels # targets/labels  # targets/labels
y.shape


## XOR decision boundary: linear vs 2-layer MLP (quick demo)

In [ ]:
rng = np.random.default_rng(0) # RNG setup
X = rng.uniform(-1, 1, size = (600, 2)) # inputs # inputs  # inputs
y = ((X[:, 0]>0) ^ (X[:, 1]>0)).astype(int) # targets/labels # targets/labels  # targets/labels
X = X + rng.normal(0, 0.15, size = X.shape) # inputs # inputs  # inputs
from sklearn.linear_model import LogisticRegression
lin = LogisticRegression().fit(X, y) # fit a linear classifier
W1 = torch.randn(2, 8, requires_grad = True) # layer 1 weights # layer 1 weights  # layer 1 weights
b1 = torch.zeros(8, requires_grad = True) # layer 1 bias # layer 1 bias  # layer 1 bias
W2 = torch.randn(8, 2, requires_grad = True) # layer 2 weights # layer 2 weights  # layer 2 weights
b2 = torch.zeros(2, requires_grad = True) # layer 2 bias # layer 2 bias  # layer 2 bias
with torch.no_grad():
    W1.mul_(0.5)
    W2.mul_(0.5)
    X_t = torch.tensor(X, dtype = torch.float32)
    y_t = torch.tensor(y, dtype = torch.long)
    for _ in range(1500):
        h = torch.relu(X_t@W1 + b1) # hidden activations  # hidden activations
        logits = h@W2 + b2 # raw model scores before softmax/sigmoid
        loss = torch.nn.functional.cross_entropy(logits, y_t) # training objective
        for p in (W1, b1, W2, b2):
            if p.grad is not None: p.grad.zero_()
            loss.backward()
            with torch.no_grad():
                for p in (W1, b1, W2, b2): p -= 0.1*p.grad
                xmin, xmax = X[:, 0].min()-0.4, X[:, 0].max()+0.4
                ymin, ymax = X[:, 1].min()-0.4, X[:, 1].max()+0.4
                xx, yy = np.meshgrid(np.linspace(xmin, xmax, 200), np.linspace(ymin, ymax, 200))
                grid = np.c_[xx.ravel(), yy.ravel()]
                with torch.no_grad():
                    h = torch.relu(torch.tensor(grid,   # hidden activations
                    dtype = torch.float32)@W1 + b1) # hidden activations
                    zz_mlp = (h@W2 + b2).argmax(dim = 1).numpy().reshape(xx.shape)
                    zz_lin = lin.predict(grid).reshape(xx.shape)
                    fig, ax = plt.subplots(1, 2, figsize = (8, 3), sharex = True,   # plotting
                    sharey = True) # plotting
                    for a, zz, title in zip(ax, [zz_lin, zz_mlp], ['Linear', '2-layer MLP']):
                        a.contourf(xx, yy, zz, levels = [-0.5, 0.5, 1.5], cmap = 'coolwarm', alpha = 0.25)
                        a.scatter(X[y==0, 0], X[y==0, 1], s = 8)
                        a.scatter(X[y==1, 0], X[y==1, 1], s = 8)
                        a.set_title(title)
                        plt.tight_layout() # plotting
                        plt.show() # plotting


<img src="https://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>
